In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import math
import seaborn as sns
import multiprocessing
from multiprocessing import Process

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
class Result:
    def __init__(self, dataset_type, random_state, test_size, algorithm_name, fit_time,
                 figure, r_square, mea, mse, rmse, k):
        self.dataset_type = dataset_type
        self.random_state = random_state
        self.test_size = test_size
        self.algorithm_name = algorithm_name
        self.fit_time = fit_time
        self.figure = figure
        self.r_square = r_square
        self.mea = mea
        self.mse = mse
        self.rmse = rmse
        self.k = k
    
    def print_object(self):
        print(f"""
        Algorithm: {self.algorithm_name}
        Dataset: {self.dataset_type}
        Test Size: {self.test_size}
        Random State: {self.random_state}
        Fit Time: {self.fit_time}
        Figure Path: {self.figure}
        R Square: {self.r_square}
        MEA: {self.mea}
        MSE: {self.mse}
        RMSE: {self.rmse}
        K: {self.k}\n\n
        """)

In [ ]:
results1 = multiprocessing.Manager().list()
results2 = multiprocessing.Manager().list()
results3 = multiprocessing.Manager().list()

In [ ]:
def runInParallel(*fns):
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        proc.append(p)
    for p in proc:
        p.join()

In [ ]:
def standard_scaler(dataset):
    columns = dataset.columns.to_list()
    scaler = StandardScaler() 
    scaled = scaler.fit_transform(dataset.to_numpy())
    return pd.DataFrame(scaled, columns=columns)

In [ ]:
def min_max_scaler(dataset):
    columns = dataset.columns.to_list()
    scaler = MinMaxScaler() 
    scaled = scaler.fit_transform(dataset.to_numpy())
    return pd.DataFrame(scaled, columns=columns)

In [ ]:
data1 = pd.read_csv('process.csv')
data2 = standard_scaler(data1)
data3 = min_max_scaler(data1)

In [ ]:
def plotGraph(y_test, y_pred, regressorName, figName):
    plt.figure(figsize=(10,10))
    plt.scatter(y_test,y_pred, c='crimson')
    plt.yscale('log')
    plt.xscale('log')

    p1 = max(max(y_pred), max(y_test))
    p2 = min(min(y_pred), min(y_test))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('True Values', fontsize=15)
    plt.ylabel('Predictions', fontsize=15)
    plt.axis('equal')
    plt.savefig(figName, format="svg")
    plt.close()

In [ ]:
def split_test(rand, size, dataset):
    y = dataset['new_cases'].values
    x = dataset.drop(['new_cases'], axis=1).values
    return train_test_split(x, y, random_state=rand, test_size=size)

In [ ]:
def liner_test(name, rand, size, X_train, X_test, y_train, y_test):
        start = datetime.datetime.now()
        
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        
        end = datetime.datetime.now()
        
        figureName = f"LN-{name}-{rand}-{size}"
        plotGraph(y_test, y_pred, 'Linear Regression', figureName)

        r_square = (1 - ((1 - metrics.r2_score(y_test, y_pred)) * (len(y_test) - 1)) / (len(y_test) - X_train.shape[1] - 1))
        mea = metrics.mean_absolute_error(y_test, y_pred)
        mse = metrics.mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        
        res = Result(name, rand, size, "LinearRegression", (end-start), figureName, r_square, mea, mse, rmse, 0)
        results1.append(res)

In [ ]:
def rand_forest(name, rand, size, X_train, X_test, y_train, y_test):
    start = datetime.datetime.now()
    
    rf = RandomForestRegressor(n_estimators = 1000 - rand, random_state = rand + 10)
    rf.fit(X_train, y_train);
    y_pred = rf.predict(X_test)
    
    end = datetime.datetime.now()
    
    figureName = f"RF-{name}-{rand}-{size}"
    plotGraph(y_test, y_pred, 'Random Forest', figureName)
    r_square = (1 - ((1 - metrics.r2_score(y_test, y_pred)) * (len(y_test) - 1)) / (len(y_test) - X_train.shape[1] - 1))
    mea = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    
    res = Result(name, rand, size, "RandomForest", (end-start), figureName, r_square, mea, mse, rmse, 0)
    results2.append(res)

In [ ]:
def knn_regression(name, rand, size, X_train, X_test, y_train, y_test):
    start = datetime.datetime.now()
    for K in range(20):
        K = K+1
        
        clf = KNeighborsRegressor(n_neighbors = K)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        end = datetime.datetime.now()
        
        figureName = f"RF-{name}-{rand}-{size}-{K}"
        plotGraph(y_test, y_pred, f"KNN with K {K}", figureName)
        
        r_square = (1 - ((1 - metrics.r2_score(y_test, y_pred)) * (len(y_test) - 1)) / (len(y_test) - X_train.shape[1] - 1))
        mea = metrics.mean_absolute_error(y_test, y_pred)
        mse = metrics.mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        
        res = Result(name, rand, size, "KNN", (end-start), figureName, r_square, mea, mse, rmse, K)
        results3.append(res)

In [ ]:
def test1():
    start = datetime.datetime.now()
    first_time = True
    for i in range(0, 11):        
        if first_time:
            rand = 0
            size = 0.05
            first_time = False
        else:
            rand = i * 5
            size = i * 0.05
        
        X_train, X_test, y_train, y_test = split_test(rand, size, data1)
        liner_test('Original', rand, size, X_train, X_test, y_train, y_test)
        rand_forest('Original', rand, size, X_train, X_test, y_train, y_test)
        knn_regression('Original', rand, size, X_train, X_test, y_train, y_test)
        
    end = datetime.datetime.now()
    print(f"\nTest1 Time: ", end-start)

In [ ]:
def test2():
    start = datetime.datetime.now()
    first_time = True
    for i in range(0, 11):
        if first_time:
            rand = 0
            size = 0.05
            first_time = False
        else:
            rand = i * 5
            size = i * 0.05
        
        X_train, X_test, y_train, y_test = split_test(rand, size, data2)
        liner_test('Standard Scaler', rand, size, X_train, X_test, y_train, y_test)
        rand_forest('Standard Scaler', rand, size, X_train, X_test, y_train, y_test)
        knn_regression('Standard Scaler', rand, size, X_train, X_test, y_train, y_test)
        
    end = datetime.datetime.now()
    print(f"\nTest2 Time: ", end-start)

In [ ]:
def test3():
    start = datetime.datetime.now()
    first_time = True
    
    for i in range(0, 11):        
        if first_time:
            rand = 0
            size = 0.05
            first_time = False
        else:
            rand = i * 5
            size = i * 0.05
            
        rand = int(rand)
        X_train, X_test, y_train, y_test = split_test(rand, size, data3)
        liner_test('MinMax Scaler', rand, size, X_train, X_test, y_train, y_test)
        rand_forest('MinMax Scaler', rand, size, X_train, X_test, y_train, y_test)
        knn_regression('MinMax Scaler', rand, size, X_train, X_test, y_train, y_test)
        
    end = datetime.datetime.now()
    print(f"\nTest3 Time: ", end-start)

In [ ]:
start = datetime.datetime.now()
print(f"Parallel start at {start}")
runInParallel(test1, test2, test3)
end = datetime.datetime.now()
print(f"\n\nParallel time: ", end-start)

In [ ]:
for item in results2:
    results3.append(item)
for item in results1:
    results3.append(item)

In [ ]:
results = list(results3)

In [ ]:
results.sort(key=lambda x: x.r_square, reverse=True)
for i in range(0, 11):
    print(f"Position {i}:")
    results[i].print_object()

In [ ]:
results.sort(key=lambda x: x.mea, reverse=False)
for i in range(0, 11):
    print(f"Position {i}:")
    results[i].print_object()

In [ ]:
results.sort(key=lambda x: x.mse, reverse=False)
for i in range(0, 11):
    print(f"Position {i}:")
    results[i].print_object()

In [ ]:
results.sort(key=lambda x: x.rmse, reverse=False)
for i in range(0, 11):
    print(f"Position {i}:")
    results[i].print_object()